In [ ]:
!pip install opendatasets

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import opendatasets as od
#5acab72028e81c5fbbddde9eb6202108
od.download("https://www.kaggle.com/datasets/xiaopengzhang12/lung-cancer-mri-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: flourisholaiya
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/xiaopengzhang12/lung-cancer-mri-images


In [ ]:
# Data paths (adjust if necessary)
data_dir = "//content/lung-cancer-mri-images/lung_cancer_MRI_dataset/train"
data_dir2 = "//content/lung-cancer-mri-images/lung_cancer_MRI_dataset/validate"
categories = ["cancer", "no_cancer"] #lung_aca", "lung_n", "lung_scc"]

# Image preprocessing parameters
img_size = (128, 128)  # Resize images to a consistent size
batch_size = 32

# Image data generation
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, img_size)
    image = image / 255.0  # Normalize pixel values
    return image

In [ ]:
image_data = []
labels = []
for category_index, category in enumerate(categories):
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        try:
            img_array = preprocess_image(os.path.join(path, img))
            image_data.append(img_array)
            labels.append(category_index)
        except Exception as e:
            print(f"Error processing image: {img}, Error: {e}")

# Convert to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

In [ ]:
image_data = []
labels = []
for category_index, category in enumerate(categories):
    # Process images from both directories in one loop
    for data_dir in [data_dir, data_dir2]:  # Iterate through both directories
        path = os.path.join(data_dir, category)
        for img in os.listdir(path):
            try:
                img_array = preprocess_image(os.path.join(path, img))
                image_data.append(img_array)
                labels.append(category_index)
            except Exception as e:
                print(f"Error processing image: {img}, Error: {e}")

# Convert to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

In [ ]:
# One-hot encode labels
labels = tf.keras.utils.to_categorical(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [ ]:
# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), # Added dropout for regularization
    layers.Dense(2, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.1) # Reduced epochs and added validation_split

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 62s 927ms/step - accuracy: 0.6870 - loss: 0.6315 - val_accuracy: 0.7467 - val_loss: 0.4406
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 82s 935ms/step - accuracy: 0.7700 - loss: 0.4326 - val_accuracy: 0.7642 - val_loss: 0.3674
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 85s 985ms/step - accuracy: 0.7988 - loss: 0.3459 - val_accuracy: 0.8210 - val_loss: 0.3489
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 77s 912ms/step - accuracy: 0.8176 - loss: 0.3351 - val_accuracy: 0.7904 - val_loss: 0.3484
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 82s 912ms/step - accuracy: 0.8291 - loss: 0.2998 - val_accuracy: 0.8166 - val_loss: 0.3191
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 57s 885ms/step - accuracy: 0.8487 - loss: 0.2918 - val_accuracy: 0.8472 - val_loss: 0.3194
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 85s 925ms/step - accuracy: 0.8673 - loss: 0.2615 - val_accuracy: 0.8384 - val_loss: 0.3315
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 81s 906ms/step - accuracy: 0.8747 - loss: 0.2295 - val_accu

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 266ms/step - accuracy: 0.8539 - loss: 0.2937
Test Loss: 0.2935
Test Accuracy: 0.8584


In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 355ms/step


In [ ]:
# Classification report and confusion matrix
print(classification_report(y_true_classes, y_pred_classes))
print(confusion_matrix(y_true_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.95      0.83      0.88       372
           1       0.74      0.92      0.82       200

    accuracy                           0.86       572
   macro avg       0.84      0.87      0.85       572
weighted avg       0.88      0.86      0.86       572

[[308  64]
 [ 17 183]]


In [ ]:
model.save("lung_cancer_model.keras")